In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

## Mandatory CUDA Hello World

In [1]:
! touch hello_world.cu
"""
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

__global__ void helloFromGPU()
{
    /*
        We can't use cout in device. Why?
        Because The GPU does not have access to standard
        output streams like std::cout, which are managed
        by the host(CPU) operating system.
    */
    printf("Hello World from GPU!\n");
}

int main()
{
    // Print from host
    cout<<"Hello World from CPU!"<<"\n";

    /*
        Launch a kernel on the GPU with one thread to
        print from GPU
    */
    helloFromGPU<<<1, 1>>>();

    // Wait for GPU to finish before accessing on host
    cudaDeviceSynchronize();

    return 0;
}
"""

'\n#include <iostream>\n#include <cuda_runtime.h>\n\nusing namespace std;\n\n__global__ void helloFromGPU()\n{\n    /*\n        We can\'t use cout in device. Why?\n        Because The GPU does not have access to standard\n        output streams like std::cout, which are managed\n        by the host(CPU) operating system.\n    */\n    printf("Hello World from GPU!\n");\n}\n\nint main()\n{\n    // Print from host\n    cout<<"Hello World from CPU!"<<"\n";\n\n    /*\n        Launch a kernel on the GPU with one thread to\n        print from GPU\n    */\n    helloFromGPU<<<1, 1>>>();\n\n    // Wait for GPU to finish before accessing on host\n    cudaDeviceSynchronize();\n\n    return 0;\n}\n'

In [ ]:
! nvcc hello_world.cu -o hello_world
! ./hello_world

Hello World from CPU!
Hello World from GPU!


In [ ]:
! touch addition.cu
"""
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__ void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  long long int N = 1000000;
  double *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0;
    y[i] = 2.0;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}
"""

In [ ]:
! nvcc addition.cu -o addition
! ./addition